In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import pandas as pd

In [ ]:
from google.cloud import bigquery

In [ ]:
client = bigquery.Client('top-moment-339420')

In [ ]:
dataset_ref = client.dataset('noaa_gsod', project='bigquery-public-data')

In [ ]:
dataset = client.get_dataset(dataset_ref)

In [ ]:
[x.table_id for x in client.list_tables(dataset)]

['gsod1929',
 'gsod1930',
 'gsod1931',
 'gsod1932',
 'gsod1933',
 'gsod1934',
 'gsod1935',
 'gsod1936',
 'gsod1937',
 'gsod1938',
 'gsod1939',
 'gsod1940',
 'gsod1941',
 'gsod1942',
 'gsod1943',
 'gsod1944',
 'gsod1945',
 'gsod1946',
 'gsod1947',
 'gsod1948',
 'gsod1949',
 'gsod1950',
 'gsod1951',
 'gsod1952',
 'gsod1953',
 'gsod1954',
 'gsod1955',
 'gsod1956',
 'gsod1957',
 'gsod1958',
 'gsod1959',
 'gsod1960',
 'gsod1961',
 'gsod1962',
 'gsod1963',
 'gsod1964',
 'gsod1965',
 'gsod1966',
 'gsod1967',
 'gsod1968',
 'gsod1969',
 'gsod1970',
 'gsod1971',
 'gsod1972',
 'gsod1973',
 'gsod1974',
 'gsod1975',
 'gsod1976',
 'gsod1977',
 'gsod1978',
 'gsod1979',
 'gsod1980',
 'gsod1981',
 'gsod1982',
 'gsod1983',
 'gsod1984',
 'gsod1985',
 'gsod1986',
 'gsod1987',
 'gsod1988',
 'gsod1989',
 'gsod1990',
 'gsod1991',
 'gsod1992',
 'gsod1993',
 'gsod1994',
 'gsod1995',
 'gsod1996',
 'gsod1997',
 'gsod1998',
 'gsod1999',
 'gsod2000',
 'gsod2001',
 'gsod2002',
 'gsod2003',
 'gsod2004',
 'gsod2005',

In [ ]:
def estimate_gigabytes_scanned(query, bq_client):
    # see https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs#configuration.dryRun
    my_job_config = bigquery.job.QueryJobConfig()
    my_job_config.dry_run = True
    my_job = bq_client.query(query, job_config=my_job_config)
    BYTES_PER_GB = 2**30
    return my_job.total_bytes_processed / BYTES_PER_GB

In [ ]:
avgTemps = '''
SELECT 
    stn, avg_temp, lat,lon, name
FROM(
    SELECT stn,AVG(data.temp) AS avg_temp
    FROM `bigquery-public-data.noaa_gsod.gsod2019` AS data
    GROUP BY stn) temp_
INNER JOIN 
    `bigquery-public-data.noaa_gsod.stations`AS stations
ON 
    temp_.stn = stations.usaf
'''

In [ ]:
estimate_gigabytes_scanned("SELECT mo, da, temp, stp, wdsp, mxpsd, gust, max, min, prcp, fog, rain_drizzle, snow_ice_pellets, hail, thunder, name, lat, lon FROM `bigquery-public-data.noaa_gsod.gsod2019` JOIN `bigquery-public-data.noaa_gsod.stations` ON `bigquery-public-data.noaa_gsod.gsod2019`.stn = `bigquery-public-data.noaa_gsod.stations`.usaf WHERE `bigquery-public-data.noaa_gsod.gsod2019`.stn IN ('150010', '150030', '150090', '150105', '150235', '150320', '150400', '150410', '150520', '150550', '150750', '150890', '151205', '151320', '151340', '151400', '151455', '151580', '151590', '151740', '152005', '152120', '152210', '152410', '152590', '152670', '152730', '152820', '152870', '152900', '152980', '152990', '153210', '153355', '153550', '153560', '153630', '153890', '154230', '154240', '154290', '154360', '154430', '154470', '154510', '154580', '154630', '154740', '154770', '154930', '150110', '150130', '150430', '150540', '150620', '150700', '150740', '150790', '150840', '150910', '150920', '150980', '151150', '151160', '151490', '151520', '151750', '151860', '151880', '151910', '152100', '152110', '152220', '152370', '152390', '152490', '152780', '152810', '152830', '152950', '153030', '153040', '153110', '153120', '153220', '153480', '153510', '153570', '153580', '153610', '153990', '154010', '154130', '154140', '154480', '154490', '154830', '154950', '154760', '150000', '150001', '150002', '150040', '150070', '150100', '150140', '150150', '150200', '150230', '150250', '150330', '150420', '150440', '150470', '150560', '150630', '150690', '150730', '150800', '150830', '150850', '150880', '150900', '150940', '150950', '150990', '151070', '151080', '151090', '151110', '151130', '151170', '151180', '151190', '151200', '151230', '151240', '151270', '151360', '151380', '151430', '151450', '151480', '151500', '151540', '151600', '151620', '151630', '151650', '151680', '151700', '151790', '151820', '151840', '151890', '151940', '151970', '151990', '152000', '152040', '152060', '152080', '152090', '152150', '152170', '152190', '152300', '152310', '152350', '152380', '152450', '152470', '152540', '152600', '152610', '152620', '152640', '152650', '152700', '152770', '152790', '152800', '152840', '152850', '152890', '152920', '152960', '152970', '153000', '153010', '153020', '153070', '153100', '153140', '153150', '153160', '153170', '153190', '153200', '153240', '153250', '153280', '153350', '153370', '153380', '153400', '153410', '153440', '153450', '153460', '153470', '153490', '153500', '153640', '153660', '153690', '153730', '153750', '153770', '153880', '153950', '154020', '154050', '154060', '154080', '154090', '154100', '154120', '154160', '154190', '154200', '154210', '154215', '154220', '154250', '154340', '154440', '154450', '154500', '154550', '154600', '154620', '154650', '154690', '154700', '154750', '154790', '154800', '154810', '154820', '154890', '154900', '154910', '154931', '154940', '154980', '153870', '153360', '154280', '153600', '154990', '153330')", client)

0.3487811693921685

In [ ]:
x = client.query("SELECT name, lat, lon, usaf FROM `bigquery-public-data.noaa_gsod.stations` WHERE country='RO'")

In [ ]:
x.result()

In [ ]:
stationList = ""

for row in x:
  stationList += "'" + row[3] + "', "

print(stationList)

'150010', '150030', '150090', '150105', '150235', '150320', '150400', '150410', '150520', '150550', '150750', '150890', '151205', '151320', '151340', '151400', '151455', '151580', '151590', '151740', '152005', '152120', '152210', '152410', '152590', '152670', '152730', '152820', '152870', '152900', '152980', '152990', '153210', '153355', '153550', '153560', '153630', '153890', '154230', '154240', '154290', '154360', '154430', '154470', '154510', '154580', '154630', '154740', '154770', '154930', '150110', '150130', '150430', '150540', '150620', '150700', '150740', '150790', '150840', '150910', '150920', '150980', '151150', '151160', '151490', '151520', '151750', '151860', '151880', '151910', '152100', '152110', '152220', '152370', '152390', '152490', '152780', '152810', '152830', '152950', '153030', '153040', '153110', '153120', '153220', '153480', '153510', '153570', '153580', '153610', '153990', '154010', '154130', '154140', '154480', '154490', '154830', '154950', '154760', '150000', 

In [ ]:
y00 = client.query("SELECT year, mo, da, temp, stp, wdsp, mxpsd, gust, max, min, prcp, fog, rain_drizzle, snow_ice_pellets, hail, thunder, name, lat, lon FROM `bigquery-public-data.noaa_gsod.gsod2017` JOIN `bigquery-public-data.noaa_gsod.stations` ON `bigquery-public-data.noaa_gsod.gsod2017`.stn = `bigquery-public-data.noaa_gsod.stations`.usaf WHERE `bigquery-public-data.noaa_gsod.gsod2017`.stn IN ('150010', '150030', '150090', '150105', '150235', '150320', '150400', '150410', '150520', '150550', '150750', '150890', '151205', '151320', '151340', '151400', '151455', '151580', '151590', '151740', '152005', '152120', '152210', '152410', '152590', '152670', '152730', '152820', '152870', '152900', '152980', '152990', '153210', '153355', '153550', '153560', '153630', '153890', '154230', '154240', '154290', '154360', '154430', '154470', '154510', '154580', '154630', '154740', '154770', '154930', '150110', '150130', '150430', '150540', '150620', '150700', '150740', '150790', '150840', '150910', '150920', '150980', '151150', '151160', '151490', '151520', '151750', '151860', '151880', '151910', '152100', '152110', '152220', '152370', '152390', '152490', '152780', '152810', '152830', '152950', '153030', '153040', '153110', '153120', '153220', '153480', '153510', '153570', '153580', '153610', '153990', '154010', '154130', '154140', '154480', '154490', '154830', '154950', '154760', '150000', '150001', '150002', '150040', '150070', '150100', '150140', '150150', '150200', '150230', '150250', '150330', '150420', '150440', '150470', '150560', '150630', '150690', '150730', '150800', '150830', '150850', '150880', '150900', '150940', '150950', '150990', '151070', '151080', '151090', '151110', '151130', '151170', '151180', '151190', '151200', '151230', '151240', '151270', '151360', '151380', '151430', '151450', '151480', '151500', '151540', '151600', '151620', '151630', '151650', '151680', '151700', '151790', '151820', '151840', '151890', '151940', '151970', '151990', '152000', '152040', '152060', '152080', '152090', '152150', '152170', '152190', '152300', '152310', '152350', '152380', '152450', '152470', '152540', '152600', '152610', '152620', '152640', '152650', '152700', '152770', '152790', '152800', '152840', '152850', '152890', '152920', '152960', '152970', '153000', '153010', '153020', '153070', '153100', '153140', '153150', '153160', '153170', '153190', '153200', '153240', '153250', '153280', '153350', '153370', '153380', '153400', '153410', '153440', '153450', '153460', '153470', '153490', '153500', '153640', '153660', '153690', '153730', '153750', '153770', '153880', '153950', '154020', '154050', '154060', '154080', '154090', '154100', '154120', '154160', '154190', '154200', '154210', '154215', '154220', '154250', '154340', '154440', '154450', '154500', '154550', '154600', '154620', '154650', '154690', '154700', '154750', '154790', '154800', '154810', '154820', '154890', '154900', '154910', '154931', '154940', '154980', '153870', '153360', '154280', '153600', '154990', '153330')")

In [ ]:
y0 = y = client.query("SELECT year, mo, da, temp, stp, wdsp, mxpsd, gust, max, min, prcp, fog, rain_drizzle, snow_ice_pellets, hail, thunder, name, lat, lon FROM `bigquery-public-data.noaa_gsod.gsod2018` JOIN `bigquery-public-data.noaa_gsod.stations` ON `bigquery-public-data.noaa_gsod.gsod2018`.stn = `bigquery-public-data.noaa_gsod.stations`.usaf WHERE `bigquery-public-data.noaa_gsod.gsod2018`.stn IN ('150010', '150030', '150090', '150105', '150235', '150320', '150400', '150410', '150520', '150550', '150750', '150890', '151205', '151320', '151340', '151400', '151455', '151580', '151590', '151740', '152005', '152120', '152210', '152410', '152590', '152670', '152730', '152820', '152870', '152900', '152980', '152990', '153210', '153355', '153550', '153560', '153630', '153890', '154230', '154240', '154290', '154360', '154430', '154470', '154510', '154580', '154630', '154740', '154770', '154930', '150110', '150130', '150430', '150540', '150620', '150700', '150740', '150790', '150840', '150910', '150920', '150980', '151150', '151160', '151490', '151520', '151750', '151860', '151880', '151910', '152100', '152110', '152220', '152370', '152390', '152490', '152780', '152810', '152830', '152950', '153030', '153040', '153110', '153120', '153220', '153480', '153510', '153570', '153580', '153610', '153990', '154010', '154130', '154140', '154480', '154490', '154830', '154950', '154760', '150000', '150001', '150002', '150040', '150070', '150100', '150140', '150150', '150200', '150230', '150250', '150330', '150420', '150440', '150470', '150560', '150630', '150690', '150730', '150800', '150830', '150850', '150880', '150900', '150940', '150950', '150990', '151070', '151080', '151090', '151110', '151130', '151170', '151180', '151190', '151200', '151230', '151240', '151270', '151360', '151380', '151430', '151450', '151480', '151500', '151540', '151600', '151620', '151630', '151650', '151680', '151700', '151790', '151820', '151840', '151890', '151940', '151970', '151990', '152000', '152040', '152060', '152080', '152090', '152150', '152170', '152190', '152300', '152310', '152350', '152380', '152450', '152470', '152540', '152600', '152610', '152620', '152640', '152650', '152700', '152770', '152790', '152800', '152840', '152850', '152890', '152920', '152960', '152970', '153000', '153010', '153020', '153070', '153100', '153140', '153150', '153160', '153170', '153190', '153200', '153240', '153250', '153280', '153350', '153370', '153380', '153400', '153410', '153440', '153450', '153460', '153470', '153490', '153500', '153640', '153660', '153690', '153730', '153750', '153770', '153880', '153950', '154020', '154050', '154060', '154080', '154090', '154100', '154120', '154160', '154190', '154200', '154210', '154215', '154220', '154250', '154340', '154440', '154450', '154500', '154550', '154600', '154620', '154650', '154690', '154700', '154750', '154790', '154800', '154810', '154820', '154890', '154900', '154910', '154931', '154940', '154980', '153870', '153360', '154280', '153600', '154990', '153330')")

In [ ]:
y = client.query("SELECT year, mo, da, temp, stp, wdsp, mxpsd, gust, max, min, prcp, fog, rain_drizzle, snow_ice_pellets, hail, thunder, name, lat, lon FROM `bigquery-public-data.noaa_gsod.gsod2019` JOIN `bigquery-public-data.noaa_gsod.stations` ON `bigquery-public-data.noaa_gsod.gsod2019`.stn = `bigquery-public-data.noaa_gsod.stations`.usaf WHERE `bigquery-public-data.noaa_gsod.gsod2019`.stn IN ('150010', '150030', '150090', '150105', '150235', '150320', '150400', '150410', '150520', '150550', '150750', '150890', '151205', '151320', '151340', '151400', '151455', '151580', '151590', '151740', '152005', '152120', '152210', '152410', '152590', '152670', '152730', '152820', '152870', '152900', '152980', '152990', '153210', '153355', '153550', '153560', '153630', '153890', '154230', '154240', '154290', '154360', '154430', '154470', '154510', '154580', '154630', '154740', '154770', '154930', '150110', '150130', '150430', '150540', '150620', '150700', '150740', '150790', '150840', '150910', '150920', '150980', '151150', '151160', '151490', '151520', '151750', '151860', '151880', '151910', '152100', '152110', '152220', '152370', '152390', '152490', '152780', '152810', '152830', '152950', '153030', '153040', '153110', '153120', '153220', '153480', '153510', '153570', '153580', '153610', '153990', '154010', '154130', '154140', '154480', '154490', '154830', '154950', '154760', '150000', '150001', '150002', '150040', '150070', '150100', '150140', '150150', '150200', '150230', '150250', '150330', '150420', '150440', '150470', '150560', '150630', '150690', '150730', '150800', '150830', '150850', '150880', '150900', '150940', '150950', '150990', '151070', '151080', '151090', '151110', '151130', '151170', '151180', '151190', '151200', '151230', '151240', '151270', '151360', '151380', '151430', '151450', '151480', '151500', '151540', '151600', '151620', '151630', '151650', '151680', '151700', '151790', '151820', '151840', '151890', '151940', '151970', '151990', '152000', '152040', '152060', '152080', '152090', '152150', '152170', '152190', '152300', '152310', '152350', '152380', '152450', '152470', '152540', '152600', '152610', '152620', '152640', '152650', '152700', '152770', '152790', '152800', '152840', '152850', '152890', '152920', '152960', '152970', '153000', '153010', '153020', '153070', '153100', '153140', '153150', '153160', '153170', '153190', '153200', '153240', '153250', '153280', '153350', '153370', '153380', '153400', '153410', '153440', '153450', '153460', '153470', '153490', '153500', '153640', '153660', '153690', '153730', '153750', '153770', '153880', '153950', '154020', '154050', '154060', '154080', '154090', '154100', '154120', '154160', '154190', '154200', '154210', '154215', '154220', '154250', '154340', '154440', '154450', '154500', '154550', '154600', '154620', '154650', '154690', '154700', '154750', '154790', '154800', '154810', '154820', '154890', '154900', '154910', '154931', '154940', '154980', '153870', '153360', '154280', '153600', '154990', '153330')")

In [ ]:
yy = client.query("SELECT year, mo, da, temp, stp, wdsp, mxpsd, gust, max, min, prcp, fog, rain_drizzle, snow_ice_pellets, hail, thunder, name, lat, lon FROM `bigquery-public-data.noaa_gsod.gsod2020` JOIN `bigquery-public-data.noaa_gsod.stations` ON `bigquery-public-data.noaa_gsod.gsod2020`.stn = `bigquery-public-data.noaa_gsod.stations`.usaf WHERE `bigquery-public-data.noaa_gsod.gsod2020`.stn IN ('150010', '150030', '150090', '150105', '150235', '150320', '150400', '150410', '150520', '150550', '150750', '150890', '151205', '151320', '151340', '151400', '151455', '151580', '151590', '151740', '152005', '152120', '152210', '152410', '152590', '152670', '152730', '152820', '152870', '152900', '152980', '152990', '153210', '153355', '153550', '153560', '153630', '153890', '154230', '154240', '154290', '154360', '154430', '154470', '154510', '154580', '154630', '154740', '154770', '154930', '150110', '150130', '150430', '150540', '150620', '150700', '150740', '150790', '150840', '150910', '150920', '150980', '151150', '151160', '151490', '151520', '151750', '151860', '151880', '151910', '152100', '152110', '152220', '152370', '152390', '152490', '152780', '152810', '152830', '152950', '153030', '153040', '153110', '153120', '153220', '153480', '153510', '153570', '153580', '153610', '153990', '154010', '154130', '154140', '154480', '154490', '154830', '154950', '154760', '150000', '150001', '150002', '150040', '150070', '150100', '150140', '150150', '150200', '150230', '150250', '150330', '150420', '150440', '150470', '150560', '150630', '150690', '150730', '150800', '150830', '150850', '150880', '150900', '150940', '150950', '150990', '151070', '151080', '151090', '151110', '151130', '151170', '151180', '151190', '151200', '151230', '151240', '151270', '151360', '151380', '151430', '151450', '151480', '151500', '151540', '151600', '151620', '151630', '151650', '151680', '151700', '151790', '151820', '151840', '151890', '151940', '151970', '151990', '152000', '152040', '152060', '152080', '152090', '152150', '152170', '152190', '152300', '152310', '152350', '152380', '152450', '152470', '152540', '152600', '152610', '152620', '152640', '152650', '152700', '152770', '152790', '152800', '152840', '152850', '152890', '152920', '152960', '152970', '153000', '153010', '153020', '153070', '153100', '153140', '153150', '153160', '153170', '153190', '153200', '153240', '153250', '153280', '153350', '153370', '153380', '153400', '153410', '153440', '153450', '153460', '153470', '153490', '153500', '153640', '153660', '153690', '153730', '153750', '153770', '153880', '153950', '154020', '154050', '154060', '154080', '154090', '154100', '154120', '154160', '154190', '154200', '154210', '154215', '154220', '154250', '154340', '154440', '154450', '154500', '154550', '154600', '154620', '154650', '154690', '154700', '154750', '154790', '154800', '154810', '154820', '154890', '154900', '154910', '154931', '154940', '154980', '153870', '153360', '154280', '153600', '154990', '153330')")

In [ ]:
yyy = yy = client.query("SELECT year, mo, da, temp, stp, wdsp, mxpsd, gust, max, min, prcp, fog, rain_drizzle, snow_ice_pellets, hail, thunder, name, lat, lon FROM `bigquery-public-data.noaa_gsod.gsod2021` JOIN `bigquery-public-data.noaa_gsod.stations` ON `bigquery-public-data.noaa_gsod.gsod2021`.stn = `bigquery-public-data.noaa_gsod.stations`.usaf WHERE `bigquery-public-data.noaa_gsod.gsod2021`.stn IN ('150010', '150030', '150090', '150105', '150235', '150320', '150400', '150410', '150520', '150550', '150750', '150890', '151205', '151320', '151340', '151400', '151455', '151580', '151590', '151740', '152005', '152120', '152210', '152410', '152590', '152670', '152730', '152820', '152870', '152900', '152980', '152990', '153210', '153355', '153550', '153560', '153630', '153890', '154230', '154240', '154290', '154360', '154430', '154470', '154510', '154580', '154630', '154740', '154770', '154930', '150110', '150130', '150430', '150540', '150620', '150700', '150740', '150790', '150840', '150910', '150920', '150980', '151150', '151160', '151490', '151520', '151750', '151860', '151880', '151910', '152100', '152110', '152220', '152370', '152390', '152490', '152780', '152810', '152830', '152950', '153030', '153040', '153110', '153120', '153220', '153480', '153510', '153570', '153580', '153610', '153990', '154010', '154130', '154140', '154480', '154490', '154830', '154950', '154760', '150000', '150001', '150002', '150040', '150070', '150100', '150140', '150150', '150200', '150230', '150250', '150330', '150420', '150440', '150470', '150560', '150630', '150690', '150730', '150800', '150830', '150850', '150880', '150900', '150940', '150950', '150990', '151070', '151080', '151090', '151110', '151130', '151170', '151180', '151190', '151200', '151230', '151240', '151270', '151360', '151380', '151430', '151450', '151480', '151500', '151540', '151600', '151620', '151630', '151650', '151680', '151700', '151790', '151820', '151840', '151890', '151940', '151970', '151990', '152000', '152040', '152060', '152080', '152090', '152150', '152170', '152190', '152300', '152310', '152350', '152380', '152450', '152470', '152540', '152600', '152610', '152620', '152640', '152650', '152700', '152770', '152790', '152800', '152840', '152850', '152890', '152920', '152960', '152970', '153000', '153010', '153020', '153070', '153100', '153140', '153150', '153160', '153170', '153190', '153200', '153240', '153250', '153280', '153350', '153370', '153380', '153400', '153410', '153440', '153450', '153460', '153470', '153490', '153500', '153640', '153660', '153690', '153730', '153750', '153770', '153880', '153950', '154020', '154050', '154060', '154080', '154090', '154100', '154120', '154160', '154190', '154200', '154210', '154215', '154220', '154250', '154340', '154440', '154450', '154500', '154550', '154600', '154620', '154650', '154690', '154700', '154750', '154790', '154800', '154810', '154820', '154890', '154900', '154910', '154931', '154940', '154980', '153870', '153360', '154280', '153600', '154990', '153330')")

In [ ]:
s = 0
for row in y.result():
  s += 1

print(s)

59347


In [ ]:
df = pd.DataFrame(columns=['year', 'mo', 'da', 'temp', 'stp', 'wdsp', 'mxpsd', 'gust', 'max', 'min', 'prcp', 'fog', 'rain_drizzle', 'snow_ice_pellets', 'hail', 'thunder', 'name', 'lat', 'lon'])

In [ ]:
len(df.columns)

19

In [ ]:
df00 = y00.to_dataframe()

In [ ]:
df0 = y0.to_dataframe()

In [ ]:
df = y.to_dataframe()

In [ ]:
df2 = yy.to_dataframe()

In [ ]:
df3 = yyy.to_dataframe()

In [ ]:
big_df = pd.concat([df00, df0, df, df2, df3])

In [ ]:
big_df

,year,mo,da,temp,stp,wdsp,mxpsd,gust,max,min,prcp,fog,rain_drizzle,snow_ice_pellets,hail,thunder,name,lat,lon
0,2017,01,16,18.8,9999.9,2.9,2.9,999.9,24.8,14.0,0.00,1,0,0,0,0,BORCEA FETESTI AIR BASE,44.383,27.717
1,2017,09,29,56.8,9999.9,17.5,20.0,28.0,59.0,55.4,0.00,0,0,0,0,0,BORCEA FETESTI AIR BASE,44.383,27.717
2,2017,01,03,32.0,9999.9,4.3,5.1,999.9,37.4,28.4,0.00,0,0,0,0,0,BORCEA FETESTI AIR BASE,44.383,27.717
3,2017,12,21,30.9,9999.9,5.6,8.0,999.9,32.0,28.4,0.00,0,0,0,0,0,BORCEA FETESTI AIR BASE,44.383,27.717
4,2017,03,08,52.9,9999.9,14.2,20.0,999.9,59.0,44.6,0.00,0,0,0,0,0,BORCEA FETESTI AIR BASE,44.383,27.717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59150,2021,03,06,10.5,814.9,5.7,7.8,999.9,25.7,6.4,0.05,1,0,1,0,0,VLADEASA 1800,46.767,22.800
59151,2021,04,15,18.8,809.4,6.3,9.7,999.9,22.5,15.6,0.21,1,0,1,0,0,VLADEASA 1800,46.767,22.800
59152,2021,05,25,52.7,817.8,20.3,27.2,999.9,57.7,42.1,0.00,1,0,0,0,1,VLADEASA 1800,46.767,22.800
59153,2021,09,09,48.1,821.9,5.6,11.7,999.9,56.1,41.9,0.00,0,0,0,0,0,VLADEASA 1800,46.767,22.800


In [ ]:
big_df.to_csv("./5_year_weather_data.csv", index=False)

In [ ]:
df[:20000]

,year,mo,da,temp,stp,wdsp,mxpsd,gust,max,min,prcp,fog,rain_drizzle,snow_ice_pellets,hail,thunder,name,lat,lon
0,2019,11,26,39.7,999.9,13.5,15.9,999.9,41.0,39.2,0.00,0,0,0,0,0,BORCEA FETESTI AIR BASE,44.383,27.717
1,2019,01,10,32.0,999.9,6.0,8.0,999.9,35.6,30.2,0.00,1,0,0,0,0,BORCEA FETESTI AIR BASE,44.383,27.717
2,2019,12,19,46.9,999.9,3.5,4.1,999.9,51.8,42.8,0.00,0,0,0,0,0,BORCEA FETESTI AIR BASE,44.383,27.717
3,2019,08,23,79.7,999.9,21.0,22.0,999.9,86.0,73.4,0.00,0,0,0,0,0,BORCEA FETESTI AIR BASE,44.383,27.717
4,2019,02,08,35.2,999.9,6.5,8.0,999.9,37.4,32.0,0.00,0,0,0,0,0,BORCEA FETESTI AIR BASE,44.383,27.717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2019,01,25,25.6,6.0,4.7,9.7,19.4,26.6,24.4,0.46,0,1,1,0,0,FOCSANI,45.700,27.200
19996,2019,04,06,48.4,9.7,2.8,5.8,999.9,60.4,39.4,0.00,0,0,0,0,0,FOCSANI,45.700,27.200
19997,2019,04,26,58.4,13.4,4.9,7.8,999.9,75.4,42.3,0.00,0,0,0,0,0,FOCSANI,45.700,27.200
19998,2019,05,14,58.5,12.1,3.7,5.8,999.9,61.7,55.9,0.00,0,1,0,0,0,FOCSANI,45.700,27.200


In [ ]:
df.to_csv("./weather_data.csv", index=False)

In [ ]:
for row in y.result():
  df = pd.concat([pd.DataFrame([[int(row[0]), int(row[1]), row[2], row[3], row[4], row[5], float(row[6]), float(row[7]), row[8], row[9], int(row[10]), int(row[11]), int(row[12]), int(row[13]), int(row[14]), row[15], row[16], row[17], row[18]]], columns=df.columns), df], ignore_index=True)

KeyboardInterrupt: ignored

In [ ]:
df

,year,mo,da,temp,stp,wdsp,mxpsd,gust,max,min,prcp,fog,rain_drizzle,snow_ice_pellets,hail,thunder,name,lat,lon


In [ ]:
for row in y.result():
  print(row[:])
  break

('11', '26', 39.7, 999.9, '13.5', '15.9', 999.9, 41.0, 39.2, 0.0, '0', '0', '0', '0', '0', 'BORCEA FETESTI AIR BASE', 44.383, 27.717)


In [ ]:
for row in x.result():
  print(row)

Row(('AVRAMENI', 48.017, 26.95, '150010'), {'name': 0, 'lat': 1, 'lon': 2, 'usaf': 3})
Row(('BOGUS ROMANIAN', 52.233, 0.467, '150030'), {'name': 0, 'lat': 1, 'lon': 2, 'usaf': 3})
Row(('DOROHOI', 47.967, 26.4, '150090'), {'name': 0, 'lat': 1, 'lon': 2, 'usaf': 3})
Row(('BAIA MARE/MAGHERUSI', 47.667, 23.667, '150105'), {'name': 0, 'lat': 1, 'lon': 2, 'usaf': 3})
Row(('SUCEAVA/SALCEA', 47.683, 26.35, '150235'), {'name': 0, 'lat': 1, 'lon': 2, 'usaf': 3})
Row(('RODNEI MOUNTAIN RNG', 47.583, 24.667, '150320'), {'name': 0, 'lat': 1, 'lon': 2, 'usaf': 3})
Row(('CIMPULUNG MOLDOVENE', 47.533, 25.567, '150400'), {'name': 0, 'lat': 1, 'lon': 2, 'usaf': 3})
Row(('RAUSENI', 47.567, 27.2, '150410'), {'name': 0, 'lat': 1, 'lon': 2, 'usaf': 3})
Row(('RARAU', 47.45, 25.567, '150520'), {'name': 0, 'lat': 1, 'lon': 2, 'usaf': 3})
Row(('RADASENI', 47.467, 26.25, '150550'), {'name': 0, 'lat': 1, 'lon': 2, 'usaf': 3})
Row(('PODUL ILOAIEI', 47.217, 27.267, '150750'), {'name': 0, 'lat': 1, 'lon': 2, 'usaf': 